- 네이버 영화 평점: https://movie.naver.com/movie/point/af/list.nhn
- 훈련 데이터: ratings_train.txt
- Test 데이터: ratings_test.txt
- 데이터: https://github.com/e9t/nsmc
    - 리뷰: document
    - 긍부정: label - 0:부정, 1:긍정

kgmyh@naver.com

## 훈련/테스트 데이터 셋 읽기

In [1]:
import pandas as pd
import numpy as np

In [7]:
train_df = pd.read_csv('ratings_train.txt', sep='\t', encoding='UTF-8')
train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [10]:
train_df.shape

(150000, 3)

In [9]:
test_df = pd.read_csv('ratings_test.txt', sep='\t', encoding='UTF-8')
test_df.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [11]:
test_df.shape

(50000, 3)

## 데이터셋 확인

In [12]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [13]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [14]:
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [15]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [18]:
train_df[train_df['document'].isna()] #제거하는 것이 좋겠다

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [20]:
# 결측치 처리 - 제거
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [21]:
train_df.isna().sum()

id          0
document    0
label       0
dtype: int64

In [22]:
test_df.isna().sum()

id          0
document    0
label       0
dtype: int64

In [23]:
# label(긍/부정) 분포
train_df['label'].value_counts()

0    75170
1    74825
Name: label, dtype: int64

In [24]:
test_df['label'].value_counts()

1    25171
0    24826
Name: label, dtype: int64

## 텍스트 전처리

In [30]:
import string, re
from konlpy.tag import Okt, Komoran
# 특정 품사의 token 추출
# N글자 이상인 token만 조회

def text_preprocessing(document):
    """document: 댓글 하나"""
    okt = Okt()
    
    #특수문자 제거
    pattern = f'[{string.punctuation}]'
    document = re.sub(pattern, ' ', document)
    
    #형태소 토큰화
    tokens = okt.morphs(document, norm=True)
    
    #불용어 제거 ....
    
    return ' '.join(tokens)

In [32]:
train_df.loc[0, 'document']

'아 더빙.. 진짜 짜증나네요 목소리'

In [33]:
text_preprocessing(train_df.loc[0, 'document'])

'아 더빙 진짜 짜증나네요 목소리'

In [34]:
text_preprocessing(train_df.loc[3, 'document'])

'교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정'

In [39]:
X_train = train_df['document'].apply(text_preprocessing)

In [40]:
X_test = test_df['document'].apply(text_preprocessing)

In [41]:
y_train = train_df['label']
y_test = test_df['label']

In [42]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((149995,), (149995,), (49997,), (49997,))

## Feature Vectorization

In [43]:
data = pd.concat([X_train, X_test])
data.shape

(199992,)

In [44]:
data.head()

0                                    아 더빙 진짜 짜증나네요 목소리
1                    흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍지 않구나
2                            너 무재 밓었 다그 래서 보는것을 추천 한 다
3                         교도소 이야기 구먼 솔직히 재미 는 없다 평점 조정
4    사이 몬페 그 의 익살스런 연기 가 돋보였던 영화 스파이더맨 에서 늙어 보이기만 했...
Name: document, dtype: object

In [45]:
data.tail()

49995    오랜 만 에 평점 로 긴 했네 ㅋㅋ 킹왕짱 쌈뽕 한 영화 를 만났습니다 강렬하게 육 쾌함
49996     의지 박약 들 이나 하는거다 탈영 은 일단 주인공 김대희 닮았고 이등병 찐 따 OOOO
49997                    그림 도 좋고 완성 도도 높았지만 보는 내내 불안하게 만든다
49998    절대 봐 서는 안 될 영화 재미 도 없고 기분 만 잡 치고 한 세트 장 에서 다 해먹 네
49999                                         마무리 는 또 왜 이래
Name: document, dtype: object

In [46]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [60]:
#cv = CountVectorizer(min_df=10)
cv = CountVectorizer(min_df=10, ngram_range=(1,3))
cv.fit(data)
train_cv = cv.transform(X_train)
test_cv = cv.transform(X_test)

In [61]:
train_cv.shape, test_cv.shape

((149995, 27932), (49997, 27932))

In [63]:
# tfidf = TfidfVectorizer(min_df=10)
tfidf = TfidfVectorizer(min_df=10, ngram_range=(1,3))
tfidf.fit(data)
train_tf = tfidf.transform(X_train)
test_tf = tfidf.transform(X_test)

In [64]:
train_tf.shape, test_tf.shape

((149995, 27932), (49997, 27932))

In [51]:
train_cv[:3].toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

## 긍부정 예측 머신러닝 모델링

In [65]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=1000, random_state=1)
lr.fit(train_cv, y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [66]:
pred_train = lr.predict(train_cv)
pred_test = lr.predict(test_cv)

In [67]:
from sklearn.metrics import accuracy_score, classification_report

accuracy_score(y_train, pred_train), accuracy_score(y_test, pred_test)

(0.8924297476582552, 0.838150289017341)

In [68]:
lr2 = LogisticRegression(max_iter=1000, random_state=1)
lr2.fit(train_tf, y_train)

LogisticRegression(max_iter=1000, random_state=1)

In [69]:
pred_train2 = lr2.predict(train_tf)
pred_test2 = lr2.predict(test_tf)

In [70]:
accuracy_score(y_train, pred_train2), accuracy_score(y_test, pred_test2)

(0.8751625054168473, 0.8422905374322459)

- cv : 
    - (0.8715623854128471, 0.8329099745984759)
    - (0.8924297476582552, 0.838150289017341) : ngram_range=(1,3)
    
- tfidf :
    - (0.8639087969598986, 0.8356501390083405)
    - (0.8751625054168473, 0.8422905374322459) : ngram_range=(1,3)